In [21]:
import os
import yaml
from pathlib import Path
import hydra

import torch
from transformers import AutoTokenizer

import sys
sys.path.append("/home/kusuma/research/codes/docsblip")
sys.path.append("../")
from src.models.docsblip_stage2 import DocsBlip


In [23]:
def init_model(ckpt_path) -> torch.nn.Module:
    config_path = os.path.join(ckpt_path, '.hydra/config.yaml')
    with open(config_path, 'r', encoding='utf-8') as f:
        config = yaml.load(f, Loader=yaml.SafeLoader)
    model = hydra.utils.instantiate(config['model'])
    model_name = ckpt_path.split('/')[-1]
    pathlist = Path(ckpt_path).glob('**/*.ckpt')
    filelist = [str(file) for file in pathlist]
    weight_path = filelist[0]
    ckpt = torch.load(weight_path, weights_only=False)#, map_location='cpu')
    weight = ckpt['state_dict']
    model.load_state_dict(weight)
    for n, p in model.named_parameters():
        p.requires_grad = False
    model.eval()
    return model

In [32]:
config['model']['_target_'] = 'docsblip.src.models.docsblip_stage2.DocsBlip'

In [34]:
config['model']

{'_target_': 'docsblip.src.models.docsblip_stage2.DocsBlip',
 'encoder_name': 'SCUT-DLVCLab/lilt-roberta-en-base',
 'decoder_name': 'meta-llama/Llama-2-7b-chat-hf',
 'max_len': 32,
 'start_lr': 1e-06,
 'ref_lr': 0.0001,
 'final_lr': 1e-05,
 'ref_wd': 0.05,
 'final_wd': 0.4,
 'dropout': 0.1,
 'warmup_steps': 2000,
 'ckpt_path': 'logs/docsblip_stage1/runs/2025-11-11_19-30-14'}

In [33]:
andmodel = init_model(ckpt_path)

InstantiationException: Error in call to target 'src.models.docsblip_stage2.DocsBlip':
FileNotFoundError(2, 'No such file or directory')